In [246]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
pd.set_option('display.max_columns', None)

In [296]:
PATH_OPERATION = "loader/operation.XLSX"
PATH_001004 = "loader/header001-004.XLSX"
PATH_070 = "loader/header070.XLSX"
DELTA_TIME = 48

In [44]:
df_operation = pd.read_excel(PATH_OPERATION)
df_h001004 = pd.read_excel(PATH_001004)
df_h070 = pd.read_excel(PATH_070)

# Traitement order h070

In [46]:

df_h070["Sales Order"] = df_h070["Sales Order"].astype(str)
df_h070["Order"] = df_h070["Order"].astype(str)
df_h070["Sales order item"] = df_h070["Sales order item"].astype(str)

# colume for order : Sales Order | Sales order item
df_h070["SO_Item"] = df_h070["Sales Order"] + df_h070["Sales order item"]
# df_h070[["Sales Order","Sales order item","SO_Item"]]

# Traitement order h001-004

In [47]:
df_h001004["Sales Order"] = df_h001004["Sales Order"].fillna(0)
df_h001004["Sales Order"] = df_h001004["Sales Order"].astype(int)
df_h001004["Sales Order"] = df_h001004["Sales Order"].astype(str)
df_h001004["Order"] = df_h001004["Order"].astype(str)
df_h001004["Sales order item"] = df_h001004["Sales order item"].astype(str)

# colume for order : Sales Order | Sales order item
df_h001004["SO_Item"] = df_h001004["Sales Order"] + df_h001004["Sales order item"]
# df_h001004[["Sales Order","Sales order item","SO_Item"]]

# Merge on so-item found eto order + flas eto

In [62]:
full_order = pd.merge(df_h001004,df_h070[["SO_Item","Order"]], on="SO_Item", how="left")
full_order = full_order.rename(columns={'Order_y': 'OF_Choice'})
full_order = full_order.rename(columns={'Order_x': 'OF_RM6'})
full_order["ETO_FLAG"] = full_order["OF_Choice"].notna()

0      False
1      False
2      False
3      False
4      False
       ...  
338    False
339    False
340    False
341    False
342    False
Name: ETO_FLAG, Length: 343, dtype: bool

# Format df operation

In [270]:
df_operation["Order"] = df_operation["Order"].astype(float)
df_operation["Activity"] = df_operation["Activity"].astype(float)
df_operation.loc[df_operation["Actual op. finish"].isna(), "Activity"] = 0
df_operation.fillna(0)

,Order,Activity,Work center,Operation short text,Capacity category,UUID,Unnamed: 6,Internal CU Number,Unloading Point,Teardown (ARUZE),Object number,Deviation reason,Change indicator,Process variant,Reqst. cost center,Requisitioner,Data changed,User Status,Number delivery notes,Counter for Demands in Opertn,No. confirm. slips,Num.splits (user),Date in Format YYYYMMSS in 8 Characters,Num.splits (sched.),Number of employees,Unnamed: 25,Work (ARBEH),Work center description,Unnamed: 28,Order sequence,Scrap,Base Unit of Measure (=/MRSS/FR_DUNIT),Scrap Quantity (MEINH),Purch. req. at fin.,Unnamed: 34,Base Quantity (MEINH),Date in Format YYYYMMSS in 8 Characters.1,Processing time (BEAZE),Req. Tracking Number,Unnamed: 39,User field unit,User field unit.1,Boolean Variable,User field unit (=USE04),User field unit (=USE05),Manual Change,Calculation key,Procurement profile,Int. initializ. carried out once,Purch.requ.item,Purchase requisition,Order probability in percent,Order Unit (=WEMEH_OPR),Purch. Order Exists,Unnamed: 54,Equity type,Object Dependencies Exist,Requirements Profile,Offset to finish,Offset to start,Start,Ref. Element PM/PS,Reference point,End,Company Code,Controlling area,Start.1,Teardown time (ARUZE),Processing time (BEAZE).1,End.1,Wait time (LIGZE),Min. duration (DAUME),Rem. Demand: No of Resources,Normal duration (DAUNE),Duration of Setup (RSTZE),Operation Set as MRS-Relevant,Duration of move (TRAZE),Queue Time (WRTZE),Id of the Time Rule,Document link,Printer,Sales Organization,Min. duration unit (=EWDME),Teardown unit (=ARUZE),Unnamed: 84,Unit/normal dur.(F) (=EWDNE),Un. forecast duratn (=PDAE),No. of Site Superv.,Qualification,Quantity Received (MEINH),Team Planning,Dispatched finish date,Dispatched finish,Date in Format YYYYMMSS in 8 Characters.2,Dispatched start date,Dispatched start,Unnamed: 96,Unit (=EHOFFB),Unnamed: 98,Unit/activity (=ILE01),Unit/activity (=ILE02),Base Unit of Measure (=/MRSS/TO_DUNIT),Unit/activity (=ILE03),Unit/activity (=ILE04),Unnamed: 104,Unit/activity (=ILE05),Unit/activity (=ILE06),Date in Format YYYYMMSS in 8 Characters.3,Unit/Process (=IPRE1),Unit for work (=ARBEH),Unnamed: 110,Processing time unit (=BEAZE),Boolean Variable.1,Minimum durat. unit (=DAUME),Normal duration unit (=DAUNE),Unit for wait time (=LIGZE),UnitForRemainProcess (=OPRE1),Unit for setup time (=RSTZE),Purchasing Group,Purchasing Document,Purchasing info rec.,Purch. Organization,Equipment,Break (ZEIER),Unit for Break Time (=ZEIER),Factory Calendar ID,Billing Plan Number,Fixed vendor,Sequence category,EarlForecstFinishDate,EarlForecstFinishTime,EarlForecstStartDate,EarlForecstStartTime,EarlFinishDateExecutn,Earl.date finish date wait,EarlFinishTimeExecutn,Earl. finish wait time,EarlStartDateTeardown,EarlStartDateProcg,EarlStartDateExecutn,EarlStartDateWait,EarlStartTimeTeardown,EarlStartTimeProcg,EarlStartTimeExecutn,EarlStartTimeWait,Ext.dat.incomplete,Functional Area,Business Area,Business Process,Inactive,Info record category,Actual op. finish,Actual start date,Actual work (ARBEH),ActFinishDateProcssg,ActFinishDateSetup,ActFinishTimeProcessg,ActFinishTimeExecutn,ActFinishTimeSetup,ActStartDateTeardown,ActStartDateProcessg,ActStartTimeTeardown,ActStartTimeProcessg,ActStartTimeExecution,JIB/JIBE Class,JIB/JIBE Subclass A,Joint venture,Joint Venture Object Type,Cstng,Cost Estimate Number,Costing Sheet,Costing Variant,No Remaining Act.,Account assignment,Cost Element,Controlling area.1,Recovery Indicator,Vendor,Cum. scrap current period (MEINH),Ind.teardown/wait time par.,Ind.continuous flow production,Ind. optional overlap,Indicator required split,Ind. required overlap,Classify operation,Fixed value,Activity type 1,Activity type 2,Activity type 3,Activity type 4,Activity type 5,Activity type 6,Wait time (LIGZE).1,Storage max. (ZEILM),Storage time proc.related (ZEILP),Wage type,Subcontracting,Wage Group,Deletion flag,LSI Ext. Processing,Material planning (WAERS),Unit for max. wait (=ZEILM),Max. problem class,Notification

In [271]:
max_op_df = df_operation.groupby('Order')["Activity"].max().reset_index()
max_op_df["Order"] = max_op_df["Order"].astype(str)
max_op_df["Order"] = max_op_df["Order"].str.replace('.0', '')
max_op_df["Activity"] = max_op_df["Activity"].astype(str)
max_op_df["Activity"] = max_op_df["Activity"].str.replace('.0', '')
max_op_df["KEY"] = max_op_df["Order"] + max_op_df["Activity"]
df_operation["Order"] = df_operation["Order"].astype(str)
df_operation["Order"] = df_operation["Order"].str.replace('.0', '')
df_operation["Activity"] = df_operation["Activity"].astype(str)
df_operation["Activity"] = df_operation["Activity"].str.replace('.0', '')
df_operation["KEY"] = df_operation["Order"] + df_operation["Activity"]

In [272]:
max_op_df_notLaunch = max_op_df[max_op_df["Activity"]=='0']
max_op_df_Launch = max_op_df[max_op_df["Activity"]!='0']
max_op_df_notLaunch

,Order,Activity,KEY
0,20219764631,0,202197646310
2,20222151680,0,202221516800
7,20223254002,0,202232540020
14,20223371048,0,202233710480
15,20223371067,0,202233710670
...,...,...,...
337,20223971221,0,202239712210
338,20223971378,0,202239713780
339,20223971528,0,202239715280
340,20223972764,0,202239727640


In [273]:
max_op_df_Launch = pd.merge(max_op_df_Launch,df_operation[["KEY","Work center","Operation short text", "Actual op. finish", "ActFinishTimeExecutn"]], on="KEY", how="left")

In [274]:
max_op_df_Launch

,Order,Activity,KEY,Work center,Operation short text,Actual op. finish,ActFinishTimeExecutn
0,20221720932,140,20221720932140,07-CTFIN,Test Report,2024-06-06,08:46:54
1,20223144299,80,2022314429980,00-HT,Controle HT,2024-06-22,03:12:12
2,20223150728,140,20223150728140,00-CTRL,Controle Final,2024-05-07,05:53:27
3,20223176671,60,2022317667160,00-HE,Controle Etancheite,2024-06-20,13:56:03
4,20223197661,60,2022319766160,00-HE,Controle Etancheite,2024-04-08,11:55:05
...,...,...,...,...,...,...,...
154,20223958081,120,20223958081120,00-EQF3,EQF3,2024-06-21,14:53:00
155,20223958083,60,2022395808360,00-HE,Controle Etancheite,2024-06-21,21:25:22
156,20223958084,60,2022395808460,00-HE,Controle Etancheite,2024-06-22,02:28:16
157,20223958324,20,2022395832420,00-SP,Montage salle propre,2024-06-20,20:03:46


# merge order/operation 

In [297]:
# add not launch
max_op_df_notLaunch = max_op_df_notLaunch.rename(columns={"Order" : "OF_RM6"})
order_notlaunch = pd.merge(full_order, max_op_df_notLaunch[["OF_RM6","Activity"]], on="OF_RM6", how="left")
order_notlaunch["Activity"] = order_notlaunch["Activity"].fillna(True)
order_notlaunch["Activity"] = order_notlaunch["Activity"].replace('0', False)
order_notlaunch["Activity"] = order_notlaunch["Activity"].astype(bool)
orders = order_notlaunch.rename(columns={"Activity" : "launchFlag"})

In [298]:
# Merge order et df_operation

max_op_df_Launch = max_op_df_Launch.rename(columns={"Order" : "OF_RM6"})
orders = pd.merge(orders, max_op_df_Launch[["OF_RM6","Activity","Work center","Operation short text","Actual op. finish","ActFinishTimeExecutn"]], on="OF_RM6", how='left')
orders["Actual op. finish"] = orders["Actual op. finish"].fillna("1900-01-01")
orders["ActFinishTimeExecutn"] = orders["ActFinishTimeExecutn"].fillna("00:00:00")

# Combiner les colonnes 'Date' et 'Heure' en un objet datetime.
# Calculer la différence entre cet objet datetime et le moment actuel.
# Ajouter une colonne 'ColonneC' avec True si la différence est supérieure à 7 heures, sinon False.
# Convertir les colonnes 'Date' et 'Heure' en une colonne datetime
orders['Datetime'] = pd.to_datetime(orders["Actual op. finish"].astype(str) + ' ' + orders["ActFinishTimeExecutn"].astype(str))

# Obtenir l'heure actuelle
now = datetime.now()

# Initialiser la colonne 'ColonneC' avec False par défaut
orders['HLFlag'] = False

# Appliquer la logique uniquement quand 'Flag' est True
orders.loc[orders['launchFlag'], 'HLFlag'] = (now - orders['Datetime']) > timedelta(hours=DELTA_TIME)



In [299]:
outpuit = ['OF_RM6','ETO_FLAG','OF_Choice','launchFlag','HLFlag','Material Number', 'Material description','Prodn Supervisor','Order quantity (GMEIN)','Basic finish date','Release date (actual)','Sales Order','Sales order item','Shortages','Route Description','Ship-to','Activity','Work center','Operation short text','Actual op. finish','ActFinishTimeExecutn','Datetime']

In [300]:
resultat = orders[outpuit]

In [301]:
resultat

,OF_RM6,ETO_FLAG,OF_Choice,launchFlag,HLFlag,Material Number,Material description,Prodn Supervisor,Order quantity (GMEIN),Basic finish date,Release date (actual),Sales Order,Sales order item,Shortages,Route Description,Ship-to,Activity,Work center,Operation short text,Actual op. finish,ActFinishTimeExecutn,Datetime
0,20223940190,False,NaN,True,True,S1B88016,LEVEL I DE-IDI I.C D.C/---/AA,1,1,2024-06-26,2024-06-19,0,0,R,NaN,NaN,40,00-CMI,Controle Montage Interne,2024-06-20,02:12:37,2024-06-20 02:12:37
1,20223940191,False,NaN,False,False,S1B88016,LEVEL I DE-IDI I.C D.C/---/AA,1,1,2024-06-26,2024-06-19,0,0,R,NaN,NaN,NaN,NaN,NaN,1900-01-01,00:00:00,1900-01-01 00:00:00
2,20223940246,False,NaN,True,False,S1B90112,LEVEL I RE-DIDI C/---/AA,1,1,2024-07-02,2024-06-21,0,0,NaN,NaN,NaN,60,00-HE,Controle Etancheite,2024-06-22,02:40:40,2024-06-22 02:40:40
3,20223940247,False,NaN,False,False,S1B90112,LEVEL I RE-DIDI C/---/AA,1,1,2024-07-02,2024-06-21,0,0,NaN,NaN,NaN,NaN,NaN,NaN,1900-01-01,00:00:00,1900-01-01 00:00:00
4,20223874630,False,NaN,False,False,DPPB000195,"RM6 HN SOI 24KV 12,5KA",2,1,2024-06-26,2024-06-20,0,0,NaN,NaN,NaN,NaN,NaN,NaN,1900-01-01,00:00:00,1900-01-01 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338,20223808526,False,NaN,True,False,GCR_RM6F_3_F_0,RM6 FREE COMBINATION 3 FCT,4,1,2024-06-24,2024-06-13,238224823,10,S,FRFR11 to IDC CROSSLOG,CFR01980,110,00-EQF2,EQF2,2024-06-22,03:44:34,2024-06-22 03:44:34
339,20223808530,False,NaN,False,False,GCR_RM6F_3_F_0,RM6 FREE COMBINATION 3 FCT,4,1,2024-06-19,2024-06-13,238185940,10,R,FRFR11 Drop to Ceelog,CHU04010,NaN,NaN,NaN,1900-01-01,00:00:00,1900-01-01 00:00:00
340,20223830096,False,NaN,False,False,GCR_RM6F_3_F_0,RM6 FREE COMBINATION 3 FCT,4,1,2024-06-24,2024-06-13,238241074,10,R,FRFR11 Drop to Ceelog,CHU04010,NaN,NaN,NaN,1900-01-01,00:00:00,1900-01-01 00:00:00
341,20223872926,False,NaN,False,False,GCR_RM6F_3_F_0,RM6 FREE COMBINATION 3 FCT,4,1,2024-06-25,2024-06-13,238212058,10,R,FRFR11 to IDC CROSSLOG,CFR01980,NaN,NaN,NaN,1900-01-01,00:00:00,1900-01-01 00:00:00


In [302]:
grouping = resultat.groupby("Activity").agg(
    WIP=("HLFlag", lambda x: (x==False).sum()),
    HL=("HLFlag", lambda x: (x==True).sum()),
    ETO=("ETO_FLAG", lambda x: (x==True).sum()),
    STD=("ETO_FLAG", lambda x: (x==False).sum()),
).reset_index()

grouping

,Activity,WIP,HL,ETO,STD
0,110,7,15,2,20
1,115,0,2,1,1
2,120,10,11,3,18
3,140,1,5,0,6
4,150,0,1,0,1
5,20,16,3,0,19
6,40,12,8,0,20
7,60,21,25,5,41
8,75,3,1,1,3
9,80,12,6,2,16


In [303]:
# Groupby sur 'Groupe' et 'Colonne1', puis compter les True/False dans 'Colonne2'
grouping = resultat.groupby(['Activity', 'Work center', 'HLFlag']).agg(
    ETO=('ETO_FLAG', lambda x: (x == True).sum()),
    STD=('ETO_FLAG', lambda x: (x == False).sum())
).reset_index()

# Réorganiser les résultats pour avoir le format souhaité
grouping = grouping.melt(id_vars=['Activity','Work center', 'HLFlag'], value_vars=['ETO', 'STD'], var_name='ETO_FLAG', value_name='Count')
grouping


,Activity,Work center,HLFlag,ETO_FLAG,Count
0,110,00-EQF2,False,ETO,1
1,110,00-EQF2,True,ETO,1
2,115,07-DET,True,ETO,1
3,120,00-EQF3,False,ETO,1
4,120,00-EQF3,True,ETO,2
5,140,00-CTRL,False,ETO,0
6,140,00-CTRL,True,ETO,0
7,140,07-CTFIN,True,ETO,0
8,150,00-EMB,True,ETO,0
9,20,00-SP,False,ETO,0


In [329]:
# Filtrer les lignes où launchFlag est True
filtered_resultat = resultat[resultat['launchFlag'] == True]

# Liste pour stocker les résultats
results = []

# Grouper par Activity et itérer sur chaque groupe
for activity, group in filtered_resultat.groupby('Activity'):
    wc = group['Work center'].iloc[0]
    WIP = 0
    WIP_ETO =0
    HL = 0
    HL_ETO = 0
    # Grouper par HLFlag et ETO_FLAG et compter les occurrences
    for flags, subgroup in group.groupby(['HLFlag', 'ETO_FLAG']):
        if flags[0]:
            WIP += len(subgroup[subgroup['ETO_FLAG']==False])
            WIP += len(subgroup[subgroup['ETO_FLAG']==True])
            WIP_ETO += len(subgroup[subgroup['ETO_FLAG']==True])
        else:
            HL += len(subgroup[subgroup['ETO_FLAG']==False])
            HL += len(subgroup[subgroup['ETO_FLAG']==True])
            HL_ETO += len(subgroup[subgroup['ETO_FLAG']==True])
    print(f"{activity:<5} {wc:<8} >>> WIP {WIP:<5} dont {WIP_ETO:<5} ETO || HL {HL:<5} dont {HL_ETO:<5} ETO")

    print()  # Ligne vide pour séparer les groupes d'Activity

110   00-EQF2  >>> WIP 15    dont 1     ETO || HL 7     dont 1     ETO

115   07-DET   >>> WIP 2     dont 1     ETO || HL 0     dont 0     ETO

120   00-EQF3  >>> WIP 11    dont 2     ETO || HL 10    dont 1     ETO

140   00-CTRL  >>> WIP 5     dont 0     ETO || HL 1     dont 0     ETO

150   00-EMB   >>> WIP 1     dont 0     ETO || HL 0     dont 0     ETO

20    00-SP    >>> WIP 3     dont 0     ETO || HL 16    dont 0     ETO

40    00-CMI   >>> WIP 8     dont 0     ETO || HL 12    dont 0     ETO

60    00-HE    >>> WIP 25    dont 4     ETO || HL 21    dont 1     ETO

75    07-NKT   >>> WIP 1     dont 0     ETO || HL 3     dont 1     ETO

80    00-HT    >>> WIP 6     dont 0     ETO || HL 12    dont 2     ETO



In [ ]:
print(f'')